### Data processor and shaper of Energy Balance Matrix.

> This module integrates the Latin America and Caribbean Energy Balance Matrix and sort out its data in a OSeMOSYS model structure. According to the naming convention used by OSeMOSYS a 3-letter country abbreviation will be adopted in front of the label as followed:

    [region][category][sector][fuel/commodity][###]

e.g. for "Crude oil extraction" kind of technology in "Argentina" the generic convention label is: `ARGMINCRU`

---
Author: Mario R. Peralta. A. <br>
School of Electrical Engineering, University of Costa Rica. <br>
The Electric Power and Energy Research Laboratory (EPERLab)

---




Example of implementation the `reslac.py` module: Filtering technologies per regions.

In [1]:
import reslac

RES = reslac.EnergyMatrix()
techs = RES.data_RES()


Retrieve input and output fuel to refineries (`REF`) technonoly, throughout contries:

In [2]:
"""Input fuels to REF convertion technology."""

in_refs = [r.in_fuels for r in techs if r.code == "REF"]
nonzeros = []
for fuels in in_refs:
    for fuel in fuels:
        if fuel.energy_PJ:
            nonzeros.append(fuel)

codes = set([n.code for n in nonzeros])
print(codes)

{'CRU'}


In [3]:
"""Output fuels to REF convertion technology."""

out_refs = [r.out_fuels for r in techs if r.code == "REF"]
nonzeros = []
for fuels in out_refs:
    for fuel in fuels:
        if fuel.energy_PJ:
            nonzeros.append(fuel)

codes = set([n.code for n in nonzeros])
print(codes)

{'LPG', 'NEN', 'KER', 'DSL', 'HFO', 'OSE', 'GAS', 'COK', 'GSL'}


Another convertion technology (blast furnace): `BOI`, first input and then output.

In [4]:
"""Input fuels to BOI convertion technology."""

in_refs = [r.in_fuels for r in techs if r.code == "BOI"]
nonzeros = []
for fuels in in_refs:
    for fuel in fuels:
        if fuel.energy_PJ:
            nonzeros.append(fuel)

codes = set([n.code for n in nonzeros])
print(codes)

{'COA001', 'COK', 'OSE'}


In [5]:
"""Output fuels to BOI convertion technology."""

out_refs = [r.out_fuels for r in techs if r.code == "BOI"]
nonzeros = []
for fuels in out_refs:
    for fuel in fuels:
        if fuel.energy_PJ:
            nonzeros.append(fuel)

codes = set([n.code for n in nonzeros])
print(codes)

{'GAS', 'COK', 'NEN', 'OSE'}


Initial RES, capacities intances only (Inicialize in zero PJ):

In [6]:
import reslac
RES = reslac.EnergyMatrix()
itecs = RES.initital_RES()
iref_arg = [t for t in itecs if t.region == "ARG"
            and t.code == "REF"]
print(iref_arg[0].in_fuels)
print(iref_arg[0].out_fuels)


[(CRU, ARG)]
[(COK, ARG), (HFO, ARG), (KER, ARG), (OSE, ARG), (LPG, ARG), (DSL, ARG), (GAS, ARG), (GSL, ARG), (NEN, ARG)]


Update fuels flow:

In [7]:
import reslac
RES = reslac.EnergyMatrix()
techs = RES.build_RES()

ref_arg = [t for t in techs if t.region == "ARG"
            and t.code == "BOI"]
print(ref_arg[0].in_fuels)
print(ref_arg[0].out_fuels)

vals_PJ = []
for f in ref_arg[0].in_fuels:
    vals_PJ.append(("In", f.code, f.energy_PJ))
for f in ref_arg[0].out_fuels:
    vals_PJ.append(("Out", f.code, f.energy_PJ))

print(vals_PJ)

[(GAS, ARG), (COK, ARG), (NEN, ARG), (COA001, ARG)]
[(GAS, ARG), (COK, ARG), (NEN, ARG)]
[('In', 'GAS', 0), ('In', 'COK', -28.31549019983542), ('In', 'NEN', 0), ('In', 'COA001', -29.415843701417185), ('Out', 'GAS', 29.09728931340275), ('Out', 'COK', 0), ('Out', 'NEN', 24.056066879955736)]


Retrieve Hydropower production in Costa Rica: `CRIPROHYD` by two methods:

1.  Filter labels base on fuel and technologies naming codes.
2. Instantiate objects and then index them from the data structure.

In [8]:
"""Method 1: Filter label."""

import reslac
res = reslac.EnergyMatrix()
techs = res.build_RES()

# Method 1: Filter code
techs_cri = [T for T in techs if T.region == "CRI"]
for T in techs_cri:
    # Filter PRO sector
    if T.code == "PRO":
        fuels = T.out_fuels
        for F in fuels:
            # Filter HYD fuel
            if F.code == "HYD":
                hyd_energy = F.energy_PJ

print("CRIPROHYD: ", hyd_energy, "PJ")


CRIPROHYD:  47.28600280326362 PJ


In [9]:
"""Method 2: Instantiate and index object."""

import reslac
res = reslac.EnergyMatrix()
techs = res.build_RES()

# Instantiate supply kind of technology
sup_tech = reslac.Primary_Tech(label="PRO",
                               region="CRI",
                               category="SUP")
# Instantiate HYD as primary fuel
fuel_hyd = reslac.Primary_Fuel(code="HYD",
                               energy=0.0,
                               region="CRI")
# Index output HYD fuel
out_prim_fuels = (techs[techs.index(sup_tech)]
                  .out_fuels)
hyd_energy = (out_prim_fuels[out_prim_fuels.index(fuel_hyd)]
              .energy_PJ)

print("CRIPROHYD: ", hyd_energy, "PJ")

CRIPROHYD:  47.28600280326362 PJ
